## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
'''自己決定MaxPooling2D放在哪裡'''
#

#卷積組合
classifier.add(Convolution2D(filters=128, kernel_size=3, activation='relu', padding='same'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=200,epochs=50)

Epoch 1/50
50000/50000 [==============================] - 14s 277us/step - loss: 3.7920 - accuracy: 0.5255
Epoch 2/50
50000/50000 [==============================] - 12s 240us/step - loss: 0.7117 - accuracy: 0.7625
Epoch 3/50
50000/50000 [==============================] - 12s 233us/step - loss: 0.4510 - accuracy: 0.8450
Epoch 4/50
50000/50000 [==============================] - 12s 234us/step - loss: 0.3359 - accuracy: 0.8844
Epoch 5/50
50000/50000 [==============================] - 12s 234us/step - loss: 0.2903 - accuracy: 0.8969
Epoch 6/50
50000/50000 [==============================] - 12s 232us/step - loss: 0.2596 - accuracy: 0.9086
Epoch 7/50
50000/50000 [==============================] - 12s 231us/step - loss: 0.2185 - accuracy: 0.9229
Epoch 8/50
50000/50000 [==============================] - 12s 231us/step - loss: 0.2047 - accuracy: 0.9288
Epoch 9/50
50000/50000 [==============================] - 12s 233us/step - loss: 0.1751 - accuracy: 0.9386
Epoch 10/50
50000/50000 [============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:


#(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
#print(input_result)
#classifier.predict(input_example)
classifier.evaluate(x_test,y_test)

10000/10000 [==============================] - 2s 237us/step


[3.691436385536194, 0.6665999889373779]

In [9]:
input_example=np.zeros(shape=(1,32,32,3))
input_example[0]=x_test[1010]; 
print(y_test[1010])
classifier.predict(input_example)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


array([[1.0000000e+00, 4.9259826e-16, 9.9035944e-12, 1.0423179e-22,
        2.2353765e-10, 2.9918563e-22, 6.2065048e-11, 2.8736726e-24,
        4.8374577e-15, 1.7272163e-18]], dtype=float32)